In [1]:
import tecplot as tp
import numpy as np
import pandas as pd
from tecplot.constant import *
from tecplot_error import *
from tecplot_axes import *

#frame初始化
def frame_init(position = (0,0),size = (9,8)):
    frame = tp.active_frame()
    frame.position=position
    frame.width = size[0]
    frame.height = size[1]
    fmaps = frame.plot().fieldmaps([dataset.zone(i) for i in range(frame.plot().num_fieldmaps)])
    fmaps.scatter.show = False #关闭原始点数据显示

#求解速度场
def velocity_field():
    '''
    数据转换函数，使用方式参考：
    二氧化碳浓度场：
    tp.data.operate.execute_equation('{CO2 (ppm)} = {Mass fraction of co2}*1E6')
    '''
    tp.data.operate.execute_equation('{Velocity (m/s)} = SQRT({X Velocity}**2 + {Y Velocity}**2 + {Z Velocity}**2)')

#创建剖切面云图
def creat_slice(variable = 'Static Pressure',
                translucency = True,shade = False,slices = True):
    frame = tp.active_frame()
    dataset = frame.dataset
    plot = frame.plot()
    plot.use_translucency = translucency
    plot.contour(0).variable = dataset.variable(variable)
    plot.show_shade = shade
    plot.show_slices = slices
    plot.slice(0).orientation = SliceSurface.XPlanes
    plot.slice(1).orientation = SliceSurface.YPlanes
    plot.slice(2).orientation = SliceSurface.ZPlanes
    slices = plot.slices(0,1,2)
    slices.show = True
    slices.edge.show = True
    slices.edge.line_thickness = 0.25
    slices.effects.use_translucency = True
    return slices #用于后续迫切面复位等操作

#重置剖切面位置
def reset_slices(variable = 'Static Pressure'):
    frame = tp.active_frame()
    plot = frame.plot()
    axes = plot.axes
    slices = creat_slice(variable=variable)
    temp = ()
    j = 0
    for each in axes:
        temp = temp + ((each.max + each.min)/2,)
        init_origin = [0,0,0]
        init_origin[j] = temp[j]
        slices[j].origin = tuple(init_origin)
        j += 1

#云图图例格式化
def legend_init(fonts = 'Time New Roman'):
    frame = tp.active_frame()
    legend = frame.plot().contour(0).legend
    legend.show = True
    legend.vertical = False
    legend.label_location = ContLegendLabelLocation.ContourLevels
    legend.position = (100,100)
    legend.box.box_type = tp.constant.TextBox.None_
    legend.number_font.typeface = fonts
    legend.number_font.size = 2.5
    legend.header.font.typeface = fonts
    legend.header.font.size = 2.5

#方向标格式化
def axes_init():
    frame = tp.active_frame()
    plot = frame.plot()
    plot.axes.orientation_axis.show = True
    plot.axes.orientation_axis.position = 7,7
    plot.axes.orientation_axis.size = 7

#单个方向剖面图重置
def reset_one_slice(direction = 'x'):
    slices = tp.active_frame().plot().slices(0,1,2)
    direction_dict = {'x':0,'y':1,'z':2}
    vec = direction_dict[direction]
    axes = tp.active_frame().plot().axes
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    axes_range = (l_axes[vec].max + l_axes[vec].min)/2
    init_origin = [0,0,0]
    init_origin[vec] = axes_range
    slices[vec].origin = tuple(init_origin)

#各方向自定义云图
def modify_slice(direction = 'x',num = 0):
    direciton_dict = {'x':0,'y':1,'z':2}
    vec = direciton_dict[direction]
    frame = tp.active_frame()
    plot = frame.plot()
    axes = plot.axes
    slices = plot.slices(0,1,2) #0,1,2分别对应x,y,z方向
    slices[vec].show = True
    init_slices = [0,0,0]
    init_slices[vec] = num
    slices[vec].origin = tuple(init_slices)
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    if num > l_axes[0].max or num < l_axes[0].min:
        reset_one_slice('x')
        print('Range: %.2f to %.2f' %(l_axes[0].min,l_axes[0].max))
        print('Now number: %.2f  Out of range of coordinates !' %(num))

#去除数据点集
def clear_extract(name = 'line',output = False):
    dataset = tp.active_frame().dataset
    while True:
        try:
            dataset.delete_zones(dataset.zone(name))
        except:
            if output:
                print('Clear complete')
                break
            break

#返回朝向剖切面
def direciton_vec(direction = 'x'):
    direciton_dict = {'x':0,'y':1,'z':2}
    if type(direction) == str:
        vec = direciton_dict[direction]
        slices = tp.active_frame().plot().slices(0,1,2)
        return slices[vec]
    elif type(direction) == list:
        res = []
        for each in direction:
            res.append(direciton_vec(each))
            return res

# 线数据提取
def line_data_extract(position = [1.67,-3.8,1.67],direction = 'x',stop = 5,step = 1,variable = 'Static Pressure',clear = True,extarct_name = 'line_extract'):
    frame = tp.active_frame()
    plot = frame.plot()
    plot.show_scatter = True
    axes = plot.axes
    if clear:
        clear_extract('line_extract')
        clear_extract('Extracted Points')
    direciton_dict = {'x':0,'y':1,'z':2}
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    position_res = []
    position_origin = position # 该处Y、Z坐标与Probe中的Y、Z坐标相反
    position_matrix = np.arange(position_origin[direciton_dict[direction]],stop,step)
    out_of_range_check(direction,position_matrix[-1])
    for i in position_matrix:
        position_origin[direciton_dict[direction]] = i
        position_res.append(position_origin.copy()) # 需要使用copy()函数，否则position_origin会被修改
    line = tp.data.extract.extract_line(position_res)
    variables = line.values(variable)
    if len(variables) != len(position_res):
        raise Exception('The number of data points is not equal to the number of variables ! %s Out of range' %(position_matrix[-1]))
    pd_charts = pd.DataFrame(columns=['Coordinate Point','Static Pressure'])
    for i in range(len(variables)):
        pd_charts.loc[i] = [position_res[i],variables[i]]
    line.name = extarct_name
    return pd_charts

#矩阵数据提取（仅支持轴向）
def matrix_data_extract(origin=[1.67, -3.8, 1.67], target={'x': 4, 'y': 4}, step={'x': 1, 'y': 1}, variable='Static Pressure', clear=False, extarct_name='matrix_extract'):
    direcition_dict = {'x': 0, 'y': 1, 'z': 2}
    direction = list(target.keys())
    target = list(target.values())
    step = list(step.values())
    position_matrix = []
    position_step = np.arange(origin[direcition_dict[direction[1]]], target[1], step[1])
    for i in position_step:
        origin[direcition_dict[direction[1]]] = i
        line_data_extract(position=origin.copy(), direction=direction[0], stop=target[0],
                          step=step[0], variable=variable, clear=clear, extarct_name=extarct_name)

In [2]:
# 初始化步骤
tp.session.connect()
frame = tp.active_frame()
# axes_transform()
velocity_field()
creat_slice(variable='Velocity (m/s)')
legend_init()
axes_init()
axes_fix()
reset_slices(variable='Velocity (m/s)')
# 数据提取
'''
数据提取示例

线数据：
line_data_extract(position = [1.67,-3.8,1.67],direction = 'x',stop = 4,step = 1,variable = 'Static Pressure')

矩阵数据：

'''

Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7600
Connection established.


"\n数据提取示例\n\n线数据：\nline_data_extract(position = [1.67,-3.8,1.67],direction = 'x',stop = 4,step = 1,variable = 'Static Pressure')\n\n矩阵数据：\n\n"

In [3]:
import inspect
import importlib # 动态导入模块(避免jupyter在更新模块时需要重启kernel)
from tecplot_plot import *
importlib.reload(inspect.getmodule(creat_streamtraces))
creat_streamtraces(replace=True,zone_name='inlet_mouth')
modify_slice(direction='z',num=0)

In [ ]:
# 俯视图
plot = tp.active_frame().plot()
plot.view.rotate_to_angles(0,0,0)
plot.axes.axis_mode = AxisMode.Independent
plot.axes.x_axis.show = True
plot.axes.y_axis.show = True
plot.axes.x_axis.min = -1
plot.axes.x_axis.max = 1
plot.axes.y_axis.min = -1
plot.axes.y_axis.max = 1
plot.axes.x_axis.scale_factor = 1
plot.axes.y_axis.scale_factor = 1
plot.axes.z_axis.scale_factor = 1
plot.slices(0,1,2).show = False
plot.view.width = 3


In [14]:
# 左视图
plot = tp.active_frame().plot()
plot.slices(1,2).show = False
plot.slice(0).effects.surface_translucency = 80
plot.view.rotate_to_angles(90,-90,0)
plot.axes.axis_mode = AxisMode.Independent
plot.axes.y_axis.show = True
plot.axes.z_axis.show = True
plot.axes.y_axis.min = -0.3
plot.axes.y_axis.max = 0.3
plot.axes.z_axis.min = -0.3
plot.axes.z_axis.max = 0.3
plot.axes.x_axis.scale_factor = 1
plot.axes.y_axis.scale_factor = 1
plot.axes.z_axis.scale_factor = 1
plot.axes.y_axis.ticks.spacing = 0.05
plot.axes.z_axis.ticks.spacing = 0.05
plot.view.position = (91,0,0)
plot.view.width = 1

In [5]:
page = tp.active_page()
delete_list = [f.name for f in tp.active_page().frames()]
del delete_list[0]
for i in delete_list:
    tp.active_page().delete_frame(page.frame(i))

In [6]:
add_frame(num=(4,3))

In [13]:
clear_extract(name='Streamtrace')

In [31]:
plot = tp.active_frame().plot()
streamtrace_extract = plot.streamtraces.extract()
ex_list = [i for i in streamtrace_extract]

In [30]:
a = tp.active_frame().dataset.variable('X_fix').values(ex_list[0])
b = tp.active_frame().dataset.variable('X_fix').values(ex_list[1])

In [37]:
res = pd.DataFrame()
for i in range(len(ex_list)):
    temp = pd.DataFrame()
    temp[f'Cloumns{i}'] = tp.active_frame().dataset.variable('X_fix').values(ex_list[i])
    res = pd.concat([res,temp],axis=1)

In [42]:
res_y = pd.DataFrame()
for i in range(len(ex_list)):
    temp = pd.DataFrame()
    temp[f'Cloumns{i}'] = tp.active_frame().dataset.variable('Y_fix').values(ex_list[i])
    res_y = pd.concat([res_y,temp],axis=1)

In [33]:
a = tp.active_frame().dataset.variable('X_fix').values(ex_list[0])

In [40]:
res

,Cloumns0,Cloumns1,Cloumns2,Cloumns3,Cloumns4,Cloumns5,Cloumns6,Cloumns7,Cloumns8,Cloumns9,...,Cloumns40,Cloumns41,Cloumns42,Cloumns43,Cloumns44,Cloumns45,Cloumns46,Cloumns47,Cloumns48,Cloumns49
0,0.008726,-0.009099,-0.005305,0.007737,-0.006714,-0.001658,0.001684,0.001684,0.008268,0.008268,...,0.000518,0.000518,-0.002177,0.003027,0.001836,0.003600,0.003600,0.002135,-0.000385,0.000777
1,0.008725,-0.009097,-0.005296,0.007737,-0.006706,-0.001656,0.001683,0.001683,0.008268,0.008268,...,0.000518,0.000518,-0.002176,0.003029,0.001836,0.003601,0.003601,0.002135,-0.000384,0.000779
2,0.008724,-0.009095,-0.005285,0.007737,-0.006695,-0.001654,0.001682,0.001682,0.008267,0.008267,...,0.000519,0.000519,-0.002175,0.003030,0.001838,0.003602,0.003602,0.002136,-0.000383,0.000781
3,0.008724,-0.009093,-0.005274,0.007737,-0.006685,-0.001651,0.001682,0.001682,0.008266,0.008266,...,0.000519,0.000519,-0.002173,0.003032,0.001839,0.003602,0.003602,0.002137,-0.000381,0.000784
4,0.008724,-0.009091,-0.005263,0.007736,-0.006676,-0.001648,0.001681,0.001681,0.008266,0.008266,...,0.000520,0.000520,-0.002171,0.003033,0.001841,0.003603,0.003603,0.002138,-0.000380,0.000786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,NaN,NaN,5.571159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458,NaN,NaN,5.571196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
459,NaN,NaN,5.571214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
460,NaN,NaN,5.571223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
res_y

,Cloumns0,Cloumns1,Cloumns2,Cloumns3,Cloumns4,Cloumns5,Cloumns6,Cloumns7,Cloumns8,Cloumns9,...,Cloumns40,Cloumns41,Cloumns42,Cloumns43,Cloumns44,Cloumns45,Cloumns46,Cloumns47,Cloumns48,Cloumns49
0,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,...,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310,0.055310
1,0.055414,0.055442,0.055420,0.055394,0.055436,0.055444,0.055421,0.055421,0.055387,0.055387,...,0.055511,0.055511,0.055546,0.055626,0.055511,0.055539,0.055539,0.055499,0.055486,0.055642
2,0.055559,0.055596,0.055553,0.055527,0.055600,0.055578,0.055635,0.055635,0.055465,0.055465,...,0.055752,0.055752,0.055783,0.055942,0.055798,0.055768,0.055768,0.055751,0.055661,0.055974
3,0.055710,0.055754,0.055687,0.055662,0.055767,0.055711,0.055849,0.055849,0.055543,0.055543,...,0.055993,0.055993,0.056020,0.056258,0.056084,0.055998,0.055998,0.056002,0.055837,0.056307
4,0.055869,0.055913,0.055824,0.055805,0.055934,0.055845,0.056065,0.056065,0.055621,0.055621,...,0.056249,0.056249,0.056278,0.056598,0.056389,0.056229,0.056229,0.056272,0.056013,0.056667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,NaN,NaN,0.109438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458,NaN,NaN,0.109438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
459,NaN,NaN,0.109438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
460,NaN,NaN,0.109438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
res.to_csv('X_fix.csv')
res_y.to_csv('Y_fix.csv')